In [1]:
import requests, re, json, datetime
import os, sys
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import uuid
from hashlib import md5
import dateparser
import pandas as pd

import sqlite3

first_time=False


def recombine_link_list(link_list):
    rlist = []
    t = ""
    s=-1
    for (url, text ,subtitle, desc, user, date ) in link_list:
        s = s + 1
        try:
            rlist.append((s, url, text, subtitle, desc, user, dateparser.parse(date).timestamp()))
        except:
            print ((s, url, text, subtitle, desc, user, dateparser.parse(date)))
            rlist.append((s, url, text, subtitle, desc, user, dateparser.parse(date)))
    return rlist

def recombine_anno_list(anno_list):
    rlist = []
    t = ""
    s=-1
    for (a,u,d) in anno_list:
        if u=="" and d=="":
            t=t+" "+a
        else:
            s=s+1
            t=(t+" "+a)
            #rlist.append((s,t.replace("\r\n", ""),u,d))
            rlist.append((s,t,u,dateparser.parse(d).timestamp()))
            t=""
    return rlist

def scrape_link_values(link_list_soup_element):
    link_url = link_list_soup_element.find('a')['href']
    link_attribution_regex = re.compile(r"\[(.*),\s([A-Z][a-z]{2}\s[\d]{1,2} [\d]{4})(?:\]|,\slast modified.*?])$")
    try:
        link_text = "".join(link_list_soup_element.find_next_sibling().find('nobr').strings)
    except:
        link_text = ""
    link_subtitle = "".join(link_list_soup_element.find('a').strings)
  

    try:
        link_desc = "".join("".join(link_list_soup_element.find_next_sibling().find('br').next_element))
    except TypeError:
        link_desc = "???"
    link_user = "".join(link_list_soup_element.find_next_sibling().find('a').strings)
    lstring = "".join(link_list_soup_element.find_next_sibling().strings)
    
    link_attribution_groups = link_attribution_regex.search("".join(link_list_soup_element.find_next_sibling().strings).strip())
    #print(link_attribution_groups)
    if link_attribution_groups is not None:
        #print(link_attribution_groups.group(1)==link_user, " user link ", link_user)
        link_date = link_attribution_groups.group(2)
    else:
        link_date = "".join(link_list_soup_element.find_next_sibling().strings)[lstring.find(link_user)+
                len(link_user)+2:lstring.find(']',lstring.find(link_user)+len(link_user))]
    try:
        link_date = datetime.strptime(
            re.search("([Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec]{3} \d{2} \d{4})", link_date).group(1), "%b %d %Y").isoformat()
    except:
        pass
        #print(link_date)
    if link_desc[-1:]=="[":
        link_desc=link_desc[:-1].strip()
    return link_url, link_text, link_subtitle, link_desc, link_user, link_date

def scrape_annotations(anno_element):
    #print(anno_element)
    anno_content = "".join(anno_element.find('font', attrs={'class':'fcs'}).strings)
    try:
        anno_user = "".join(anno_element.find('td', attrs={'class':'fcs'}).find('a').strings)
        #anno_date = datetime.datetime.now()
        anno_date = "".join(anno_element.find('td', attrs={'class':'fcs'}).strings)[-11:]
    except:
        anno_user = ""                        
        anno_date = ""
    return anno_content, anno_user, anno_date



def get_links(s, url):
    r = s.get (url)
    page_links_regex = re.compile("<a class=\"(?:newidea|oldidea)\" href=\"(/idea/.*?)\"")
    link_harvest = [urljoin(url,l).split("#")[0] for l in page_links_regex.findall(r.text)]
    return link_harvest

def idea_components(hb_link, start_timestamp):
    l=hb_link
    r = s.get(l)
    fetch_time=datetime.datetime.now().timestamp()
    update_since = start_timestamp
    soup=bs(r.text,"html")
    mainpanel = soup.find('td', attrs={'class':'mainpanel'})
    idea_header = mainpanel.findAll('table')[2]
    title = str("".join(idea_header.find('a', attrs={'name':'idea'}).strings))
    fetch_id = str(uuid.uuid4())
    description = "".join(mainpanel.find('font', attrs={'class':'fcl'}).strings)
    #votes = self.getvotes("".join(mainpanel.find('td', attrs={'class':'controls'}).find('td', attrs={'valign':'top', 'align':'center'}).strings).replace("(","").replace(")","").split(","))
    copy = str("".join(idea_header.find('div', attrs={'class':'copy'}).strings))
    (user, text_date) = ( n.strip() for n in str("".join(idea_header.find('td', attrs={'class':'fcm'}).strings)).split(","))
    #idate = datetime.datetime.strptime(text_date, "%b %d %Y").isoformat()
    idate=dateparser.parse(text_date).timestamp()
    links = recombine_link_list([scrape_link_values(n) for n in idea_header.findAll('font', attrs={'class':'fcm'})])
    annos = recombine_anno_list([scrape_annotations(n) for n in idea_header.next_siblings if n.name=='table'])
    #print("".join([str(j) for j in [title, description, copy, user, idate, links, annos]]).encode("utf-8"))
    ihash = md5("".join([str(j) for j in [title, description, copy, user, idate, links, annos]]).encode("utf-8")).hexdigest()
    return {
                 "fetch_id" : fetch_id,
                 "url" : l, 
                 "hash" : ihash,
                 "title":title, 
                 "description" : description, 
                 "copy" : copy, 
                 "user" : user, 
                 "idea_date" : idate, 
                 "links": links, 
                 "annos" : annos,
                 "fetch_date" : fetch_time,
                 "update_since" : start_timestamp
            }


# SQLite requires dates be converted according to some convention - here we'll use integer seconds since epoch
# or whatever is convenient.
# Also, we have a multi-table structure, since annos and links are collections of records themselves.
# So the structure looks like:

#   +--------------------+
#   |  idea_fetch        |
#   +--------------------+
#   |  fetch_id (pk)     |
#   |  url               |
#   |  hash              |
#   |  title             |
#   |  description       |
#   |  copy              |
#   |  user              |
#   |  idea_date         |
#   |  fetch_date        |
#   +--------------------+


def sql_create_schema(conn,first_time=False):
    if first_time:
        c = conn.cursor()
        c.execute( """DROP TABLE idea_fetch""")
        c.close()
        c = conn.cursor()
        c.execute( """CREATE TABLE idea_fetch
                    (   fetch_id text,
                        url text, 
                        hash text, 
                        title text, 
                        description text, 
                        copy text, 
                        user text, 
                        idea_date real, 
                        fetch_date real,
                        update_since real)""")
        c.close()
        c = conn.cursor()
        c.execute( """DROP TABLE anno_fetch""")
        c.close()
        c = conn.cursor()
        c.execute( """CREATE TABLE anno_fetch
                    (   fetch_id text,
                        anno_seq integer, 
                        anno_text text, 
                        anno_user text, 
                        anno_date real
                        )""")
        c.close()
        c = conn.cursor()
        c.execute( """DROP TABLE link_fetch""")
        c.close()
        c = conn.cursor()
        c.execute( """CREATE TABLE link_fetch
                    (   fetch_id text,
                        link_seq integer, 
                        link_url text, 
                        link_rickroll text, 
                        link_text text, 
                        link_anno text,
                        link_user text, 
                        link_date real
                        )""")
        c.close()

        return True
    

def store_fetch_record(c,record):
    idea_insert_sql = """INSERT INTO idea_fetch VALUES
                        ( ?, ?, ?, ?, ?, ?, ?, ?, ? )"""
    anno_insert_sql = """INSERT INTO anno_fetch VALUES
                        ( ?, ?, ?, ?, ? )"""
    link_insert_sql = """INSERT INTO link_fetch VALUES
                    ( ?, ?, ?, ?, ?, ?, ?, ? )"""
    links_pk = uuid.uuid4()
    annos_pk = uuid.uuid4()
    idea_values = [record["fetch_id"], 
                   record["url"], 
                   record["hash"], 
                   record["title"], 
                   record["description"], 
                   record["copy"], 
                   record["user"], 
                   record["idea_date"], 
                   record["fetch_date"]]
    #for e,v in enumerate(idea_values):
    #    print(e,v)
    c.execute(idea_insert_sql, idea_values)
    for anno in record['annos']:
        anno_values = [record["fetch_id"], 
                       anno[0], 
                       anno[1], 
                       anno[2], 
                       anno[3]]
        c.execute(anno_insert_sql, anno_values)
    
    for link in record['links']:
        link_values = [record["fetch_id"], 
                       link[0], 
                       link[1], 
                       link[2], 
                       link[3],
                       link[4],
                       link[5],
                       link[6]]
        c.execute(link_insert_sql, link_values)

def query_to_recordset(connection, query, parameters=None):
    c = connection.cursor()
    if parameters is not None:
        rs = c.execute(query, parameters)
    else:
        rs = c.execute(query)
    schema = rs.description
    return_set = []
    for r in rs:
        return_set.append({schema[e][0]:r[e] for e in range(0,len(schema))})
    rs.close()
    c.close()
    return return_set


In [2]:
print (first_time)

True


In [3]:
# If an idea component is "novel" i.e. it has a hash that's not on file, then it can be saved for posterity
# It also qualifies for a review for any content that matches the search criteria. 
# The details of successful searches are then logged independently in such a way that they can be used to 
# filter out repeat matches. 

# What should be the logging mechanism? SQLlite probably. Makes sense to create a database to host and persist
# the content. 
conn=None
c=None
conn = sqlite3.connect('hb_records.db')

In [296]:

now = datetime.datetime.now().timestamp()

# This makes use of a search url, returning all the ideas posted in some time period (defaulted to a day)
a_day = 86400
if first_time == True:
    days_since_start = (datetime.datetime.now() - datetime.datetime(2020,10,17) ).days
    start_timestamp = now-(a_day * days_since_start)
    guess_m = 500
else:
    start_timestamp = now-(a_day * 1)
    guess_m = 100
    

t_minus = int(now - start_timestamp)

print(t_minus)
url = "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:dn={m}:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(m=guess_m,t_minus=t_minus)
#url="https://www.halfbakery.com/view/ftm=r5356800:s=Qr:d=irq:do=100:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions"
#url="https://www.halfbakery.com/view/ftm=r5356800:s=Qr:d=irq:do=200:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions"
print (url)
s = requests.Session()
contents = []
link_harvest = get_links(s, url)
for l in link_harvest:
    contents.append(idea_components(l,start_timestamp))
lindex = [c['url'] for c in contents]    
#conn.row_factory = sqlite3.Row
# If this is the first time running, then we need to create the schema
if first_time is True:
    first_time = False
    sql_create_schema(conn, first_time)
print (lindex)

86400
https://www.halfbakery.com/view/ftm=r86400:s=Qr:d=irq:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions
['https://www.halfbakery.com/idea/_22Unedited_20documentary_22_20movie', 'https://www.halfbakery.com/idea/epigenetic_20epicureans', 'https://www.halfbakery.com/idea/light', 'https://www.halfbakery.com/idea/Completely_20Realistic_20Fake_20Candle', 'https://www.halfbakery.com/idea/Plaid_20conductor_20(Redundant_20Array_20of_20Independent_20Conductors)', 'https://www.halfbakery.com/idea/Colouring_20Cook_20Book', 'https://www.halfbakery.com/idea/Merry_20Christmas', 'https://www.halfbakery.com/idea/F_fcrst_20annual_20HalfBakery_20_93Wo_20ist_20der_20F_fchrer_20_3f_94_20programming_20competition_2e', 'https://www.halfbakery.com/idea/10_20Gbps_20ethernet_20using_20USB3', 'https://www.halfbakery.com/idea/The_20universal_20paradox_20of_20infinity', 'https://www.halfbakery.com/idea/Invisible_20Tree', 'https://www.halfbakery.com/i

In [297]:

c = conn.cursor()
retrieve_latest_sql = """
                select i.url, i.hash, i.fetch_date 
                from (
                    select url, max(fetch_date) max_fetch_date
                    from (
                        select url, fetch_date
                         from idea_fetch
                         where url in ({in_list}))
                    group by url) as latest_v
                    join idea_fetch i on 
                    i.url = latest_v.url and
                    i.fetch_date = latest_v.max_fetch_date 
                """.format(in_list = ",".join(["?" for l in lindex]))

rs = c.execute(retrieve_latest_sql, lindex)
r_cols = rs.description
content_filter=[]
for r in rs:
    lindex_i = lindex.index(r[0])
    if contents[lindex_i]['hash']==r[1]:
        print ("Hashmatch - no update")
        content_filter.append(r[0])
    else:
        print ("Hashfail - got update")
        
rs.close()
c.close()

save_list = list(set(lindex).difference(set(content_filter)))

for c in contents:
    if c['url'] in save_list:
        store_fetch_record(conn, c)
        print("Saving", c['url'])

Hashfail - got update
Hashfail - got update
Hashmatch - no update
Hashmatch - no update
Hashfail - got update
Hashmatch - no update
Hashmatch - no update
Hashmatch - no update
Hashfail - got update
Hashfail - got update
Hashfail - got update
Hashfail - got update
Saving https://www.halfbakery.com/idea/_22Unedited_20documentary_22_20movie
Saving https://www.halfbakery.com/idea/epigenetic_20epicureans
Saving https://www.halfbakery.com/idea/light
Saving https://www.halfbakery.com/idea/Completely_20Realistic_20Fake_20Candle
Saving https://www.halfbakery.com/idea/Plaid_20conductor_20(Redundant_20Array_20of_20Independent_20Conductors)
Saving https://www.halfbakery.com/idea/Colouring_20Cook_20Book
Saving https://www.halfbakery.com/idea/Merry_20Christmas
Saving https://www.halfbakery.com/idea/F_fcrst_20annual_20HalfBakery_20_93Wo_20ist_20der_20F_fchrer_20_3f_94_20programming_20competition_2e
Saving https://www.halfbakery.com/idea/10_20Gbps_20ethernet_20using_20USB3


In [298]:
len(lindex)

14

In [299]:
c = conn.cursor()
rs = c.execute("""select url, hash, fetch_date
                from idea_fetch 
                order by url, fetch_date """)
r_cols = rs.description

for r in rs:
    #print ( [(r_cols[e][0], r[e]) for e,v in enumerate(r)] )
    print ( [(r[e]) for e,v in enumerate(r)] )
rs.close()
c.close()

['https://www.halfbakery.com/idea/$300House', '2e52457700a0fcfac069003e2152a01f', 1608310495.925205]
['https://www.halfbakery.com/idea/10_20Gbps_20ethernet_20using_20USB3', '7c7987c36c1a3804545d0404da6744cb', 1608340321.781092]
['https://www.halfbakery.com/idea/2020_20Trick-or-Treater_20Deterrent', '80bf9bdeceb36ffd43dee8ca6bd16f91', 1608310498.072394]
['https://www.halfbakery.com/idea/30-300_25_20better_20foam_20earplugs', '738b9c2b6944c5d19d4d7af69ce52361', 1608310288.607035]
['https://www.halfbakery.com/idea/737_20Max(imum_20impact)', 'bc8d09272f25aec149fc490ddd9b5629', 1608310421.688139]
['https://www.halfbakery.com/idea/9_20Billion_20Names_20of_20Billionaires', '9eec16ee2284b61bbe0670ad90dceb34', 1608310514.918664]
['https://www.halfbakery.com/idea/A_20Haiku_20Sealed_20My_20Abdomen', '3f70183a95490b89de4e99f6d5ecb0a6', 1608310485.44931]
['https://www.halfbakery.com/idea/A_20Pristine_20Idea', 'f9eeb2ae6e8dc1912813ffb49653d36e', 1608310516.341741]
['https://www.halfbakery.com/idea/A

In [300]:
# Pick a search date and retrieve all the latest versions of chached content collected since that date. 
search_date = datetime.datetime.now().timestamp()-(86400*0.1)

c = conn.cursor()

# This does a string replace, rather than a strict parameter-style query, because
# I can't get the wretched parameterised one to work without reporting an unmatched datatype
# error. Suspect it's due to an error in the schema. 
retrieve_latest_sql = """
                select i.url, i.hash, i.fetch_date 
                from (
                    select url, max(fetch_date) max_fetch_date
                    from (
                        select url, fetch_date
                         from idea_fetch
                         where fetch_date > {search_date})
                    group by url) as latest_v
                    join idea_fetch i on 
                    i.url = latest_v.url and
                    i.fetch_date = latest_v.max_fetch_date 
                    order by i.fetch_date
                """.format(search_date=search_date)

print(search_date)

# The query returns the collection of ideas that's been most recently fetched in the last time period. 
# If they've been fetched, then they would have been up on the "recent" search list, but it's not necessarily
# the case that they were edited at this time
# The timestamp used as an effective search criterea could be inferred from the search query - and used to
# define bounds on the time between which the update happened. 
idea_set = query_to_recordset(conn, retrieve_latest_sql)


1608331684.371508


In [301]:
idea_set

[{'url': 'https://www.halfbakery.com/idea/_22Unedited_20documentary_22_20movie',
  'hash': '755b55cbf19fcbf9ee62c8d96a82cb99',
  'fetch_date': 1608340315.690002},
 {'url': 'https://www.halfbakery.com/idea/epigenetic_20epicureans',
  'hash': 'c9aa4ef6da408bce2539a2d83ac4ac85',
  'fetch_date': 1608340315.981114},
 {'url': 'https://www.halfbakery.com/idea/light',
  'hash': 'b94e4bcf88fd2164e5f4fa6e6b33b85c',
  'fetch_date': 1608340316.275138},
 {'url': 'https://www.halfbakery.com/idea/Completely_20Realistic_20Fake_20Candle',
  'hash': '0a1f0238e4a057e9fb692df487aeaf9d',
  'fetch_date': 1608340317.761142},
 {'url': 'https://www.halfbakery.com/idea/Plaid_20conductor_20(Redundant_20Array_20of_20Independent_20Conductors)',
  'hash': '0ce9aad08597ccaaa231c9c92cc4b987',
  'fetch_date': 1608340318.108216},
 {'url': 'https://www.halfbakery.com/idea/Colouring_20Cook_20Book',
  'hash': '6b7459c44af937e23baebcde12646028',
  'fetch_date': 1608340318.465579},
 {'url': 'https://www.halfbakery.com/idea/

In [302]:
len(idea_set)

9

In [303]:
# Test for bad link dates
sql="""select i.fetch_id, url, hash, link_text, link_anno, link_user, link_date from 
        link_fetch l
        left join idea_fetch i on i.fetch_id=l.fetch_id
        where link_date is null
        """
ds = query_to_recordset(conn, sql)
pd.DataFrame(ds)#['url'].values

""


In [304]:
# Test for bad link dates
sql="""select count(*) from idea_fetch
        """
ds = query_to_recordset(conn, sql)
pd.DataFrame(ds)#['url'].values

,count(*)
0,293


In [305]:
#idea_components("https://www.halfbakery.com/idea/Film_20Noir_20Home", 12345)

In [306]:
# Build a view of attributable contributions - ideas, links and annotations - 
# flattened into a single table containing 
# url, fetch_id, date, user, contribution_type and text

get_ideas_sql="""
                select url, i.fetch_id, idea_date date, user, "idea" ctype, copy text  
                from 
                idea_fetch i
                join  (select i.fetch_id from 
                        (select fetch_id, url, max(fetch_date) max_fetch_date
                        from (
                        select fetch_id, url, fetch_date
                        from idea_fetch
                        where fetch_date > {search_date})
                        group by fetch_id, url) as latest_v
                        join idea_fetch i on 
                        i.url = latest_v.url and
                        i.fetch_date = latest_v.max_fetch_date 
                        order by i.fetch_date) as latest
                        on i.fetch_id = latest.fetch_id
                    
""".format(search_date=search_date)

ds = query_to_recordset(conn, get_ideas_sql)
ideas_df=pd.DataFrame(ds)#['url'].values

In [307]:
# Build a view of attributable contributions - ideas, links and annotations - 
# flattened into a single table containing 
# url, fetch_id, date, user, contribution_type and text

get_annos_sql="""
                
                
                select i.url, a.fetch_id, anno_date date, anno_user user, "anno" ctype, anno_text text  
                from 
                anno_fetch a
                join idea_fetch i on a.fetch_id = i.fetch_id
                join  (select i.fetch_id from 
                        (select fetch_id, url, max(fetch_date) max_fetch_date
                        from (
                        select fetch_id, url, fetch_date
                        from idea_fetch
                        where fetch_date > {search_date})
                        group by fetch_id, url) as latest_v
                        join idea_fetch i on 
                        i.url = latest_v.url and
                        i.fetch_date = latest_v.max_fetch_date 
                        order by i.fetch_date) as latest
                        on i.fetch_id = latest.fetch_id
                    
""".format(search_date=search_date)

ds = query_to_recordset(conn, get_annos_sql)
annos_df=pd.DataFrame(ds)#['url'].values

In [308]:
# Build a view of attributable contributions - ideas, links and annotations - 
# flattened into a single table containing 
# url, fetch_id, date, user, contribution_type and text

get_links_sql="""
                
                
                select i.url, l.fetch_id, link_date date, link_user user, "link" ctype, link_text || " " || link_anno text  
                from 
                link_fetch l
                join idea_fetch i on l.fetch_id = i.fetch_id
                join  (select i.fetch_id from 
                        (select fetch_id, url, max(fetch_date) max_fetch_date
                        from (
                        select fetch_id, url, fetch_date
                        from idea_fetch
                        where fetch_date > {search_date})
                        group by fetch_id, url) as latest_v
                        join idea_fetch i on 
                        i.url = latest_v.url and
                        i.fetch_date = latest_v.max_fetch_date 
                        order by i.fetch_date) as latest
                        on i.fetch_id = latest.fetch_id
                    
""".format(search_date=search_date)

ds = query_to_recordset(conn, get_links_sql)
links_df= pd.DataFrame(ds)#['url'].values

In [309]:
contribution_df = pd.concat([ideas_df, annos_df, links_df], axis=0)

In [310]:
match_d = {"Hitler" : [re.compile(r"(?i)\b(hitler)\b")]}
def multi_matcher(text, match_d):
    matches={}
    wbs=[]
    wb_regex=re.compile(r"(\b)")
    for m in wb_regex.finditer(text):
        if m is not None:
            wbs.append(m.span()[0])
    for k,v in match_d.items():
        matches[k]=[]
        for vv in v:
            m = vv.finditer(text)
            if m is not None:
                for mg in m:
                    sp=mg.span()
                    try:
                        start_pos = [w for w in wbs if w < sp[0]-10][-1]
                    except IndexError:
                        start_pos = 0   
                    try:
                        end_pos = [w for w in wbs if w > sp[1]+10][0]
                    except IndexError:
                        end_pos = len(text)
                    matches[k].append ((sp, "//" + text[start_pos:end_pos] + "//"))
    for k in matches.keys():
        if matches[k]==[]:
            matches[k]=None
    return matches

In [311]:
ttext = "Hitler had a little dog, it's teeth caused dresses to fit her. And everywhere that Hitler went, she called it Adolph Hitler."

In [312]:
multi_matcher(ttext, match_d)

{'Hitler': [((0, 6), '//Hitler had a little//'),
  ((83, 89), '//everywhere that Hitler went, she //'),
  ((117, 123), '// it Adolph Hitler.//')]}

In [313]:
hmatch = contribution_df['text'].apply(lambda x : multi_matcher(x, match_d))

In [314]:
contribution_df["matches"] = hmatch[hmatch.apply(lambda x : len(x)>0)]

In [315]:
contribution_df[contribution_df["matches"].apply(lambda x : x["Hitler"] is not None)].sort_values(by="date")

,url,fetch_id,date,user,ctype,text,matches
504,https://www.halfbakery.com/idea/F_fcrst_20annu...,3ea85d15-dbc3-4fb5-a0b7-a70ffabf5c48,1607990400,pocmloc,anno,I think there should be a Hitler category,"{'Hitler': [((27, 33), '//should be a Hitler c..."
11,https://www.halfbakery.com/idea/F_fcrst_20annu...,3ea85d15-dbc3-4fb5-a0b7-a70ffabf5c48,1607990400,8th of 7,link,Days Since Hitler Was Mentioned Here The genes...,"{'Hitler': [((11, 17), '//Days Since Hitler Wa..."
